In [1]:
import re
import os
import pandas as pd
from getpass import getpass
from playwright.async_api import async_playwright

APPLE_HOME = "https://www.apple.com/jp/"
CSV_PATH = "../some-data/Book1.csv"

# 你在 UI.Vision 中固定点击的元素 id（保持一致）
BTN_UPDATE_PICKUP = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact"
INP_LASTNAME     = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName"
INP_FIRSTNAME    = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName"
BTN_SUBMIT       = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit"

In [4]:
import re
import os
import time
import pandas as pd
from getpass import getpass
from playwright.async_api import async_playwright

APPLE_HOME = "https://www.apple.com/jp/"
CSV_PATH = "../some-data/Book10.csv"

# 你在 UI.Vision 中固定点击的元素 id（保持一致）
BTN_UPDATE_PICKUP = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact"
INP_LASTNAME     = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName"
INP_FIRSTNAME    = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName"
BTN_SUBMIT       = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit"

def split_name_jp(fullname: str):
    """
    将 '神原　龍一' / '神原 龍一' 这类名字拆成 (last, first)
    """
    if not fullname or not isinstance(fullname, str):
        return "", ""
    s = fullname.replace("　", " ").strip()
    parts = [p for p in re.split(r"\s+", s) if p]
    if len(parts) >= 2:
        return parts[0], parts[1]
    return parts[0], ""  # 只有姓

async def maybe_switch_to_popup_or_same_page(context, current_page, click_coro, wait_ms=1200):
    """
    Apple 的“订单”入口有时同页跳转，有时会开新页。
    这里用“点击前后 pages 数量变化”来判断。
    """
    before = len(context.pages)
    await click_coro
    await current_page.wait_for_timeout(wait_ms)
    after = len(context.pages)
    return context.pages[-1] if after > before else current_page

async def sign_in_if_needed(page, apple_id: str, password: str):
    """
    订单页可能出现 iframe 登录组件（UI.Vision 的 selectFrame index=0）。
    如果发现 account_name_text_field，则执行登录；否则认为已登录。
    """
    # 等页面稍稳定
    await page.wait_for_load_state("domcontentloaded")

    # 找第一个 iframe，并尝试在其中定位登录输入框
    frame_loc = page.frame_locator("iframe").first
    account_field = frame_loc.locator("#account_name_text_field")

    try:
        await account_field.wait_for(timeout=6000)
    except:
        return  # 没看到登录框，认为无需登录（可能已登录）

    # 输入账号
    await account_field.fill(apple_id)
    await frame_loc.locator("#sign-in").click()
    time.sleep(10)

    # 输入密码
    await frame_loc.locator("#password_text_field").wait_for(timeout=15000)
    await frame_loc.locator("#password_text_field").fill(password)
    time.sleep(10)
    await frame_loc.locator("#sign-in").click()

    # 2FA/跳转等待：给足时间，让你手动完成
    print("如果出现 2FA 验证，请在浏览器里完成后回到这里按回车继续……")
    try:
        # 尝试等待订单列表容器出现（你的 UI.Vision 里后续用到 #rs-container）
        await page.wait_for_selector("#rs-container", timeout=180000)
    except:
        input("未自动检测到订单列表。请确认已登录并进入订单页，然后按回车继续：")

async def open_orders_page(page):
    """
    对应 UI.Vision:
      click bag -> click orders
    """
    await page.goto(APPLE_HOME, wait_until="domcontentloaded")

    # 打开购物袋菜单
    await page.locator("#globalnav-menubutton-link-bag").click()

    # 订单入口：不同时间文案可能不同，这里用“包含 订单/ご注文/Orders”做兼容
    bag_menu = page.locator("#globalnav-submenu-bag")
    orders_link = bag_menu.get_by_role(
        "link",
        name=re.compile(r"(ご注文|注文|Orders|注文状況|ご注文状況)")
    ).first

    await orders_link.wait_for(timeout=15000)
    return await maybe_switch_to_popup_or_same_page(page.context, page, orders_link.click())

async def open_target_order(page, ordernumber: str | None, product_fallback: str = "iPhone 17 Pro"):
    """
    对应 UI.Vision:
      click linkText=iPhone 17 Pro
    但这里优先用订单号打开（更稳），没有订单号就回退商品名。
    """
    await page.wait_for_load_state("domcontentloaded")

    # 优先：订单号
    if ordernumber and isinstance(ordernumber, str) and ordernumber.strip():
        on = ordernumber.strip()
        # 常见页面里订单号附近会有可点击区域；这里尽量找“包含订单号的链接”
        link = page.get_by_role("link", name=re.compile(re.escape(on))).first
        try:
            await link.wait_for(timeout=10000)
            await link.click()
            return
        except:
            # 再尝试：直接点包含订单号的文本（有时文本在 card 内）
            txt = page.get_by_text(on).first
            await txt.wait_for(timeout=10000)
            await txt.click()
            return

    # 回退：商品名
    prod_link = page.get_by_role("link", name=re.compile(re.escape(product_fallback))).first
    await prod_link.wait_for(timeout=15000)
    await prod_link.click()

async def update_pickup_contact(page, new_last: str, new_first: str):
    """
    对应 UI.Vision:
      click updatePickupContact -> type last/first -> submit
    """
    await page.wait_for_load_state("domcontentloaded")

    # 打开修改弹窗/区域
    await page.locator(f"#{BTN_UPDATE_PICKUP}").wait_for(timeout=20000)
    await page.locator(f"#{BTN_UPDATE_PICKUP}").click()

    # 填写姓/名
    await page.locator(f"#{INP_LASTNAME}").wait_for(timeout=15000)
    await page.locator(f"#{INP_LASTNAME}").fill(new_last)

    await page.locator(f"#{INP_FIRSTNAME}").wait_for(timeout=15000)
    await page.locator(f"#{INP_FIRSTNAME}").fill(new_first)

    # 提交
    await page.locator(f"#{BTN_SUBMIT}").click()

    # 等待提交后的页面/提示变化（保守等待）
    await page.wait_for_timeout(1500)

async def sign_out(page):
    """
    对应 UI.Vision:
      click bag -> click sign out
    """
    await page.locator("#globalnav-menubutton-link-bag").click()

    bag_menu = page.locator("#globalnav-submenu-bag")
    signout_link = bag_menu.get_by_role(
        "link",
        name=re.compile(r"(サインアウト|Sign Out|ログアウト)")
    ).first

    try:
        await signout_link.wait_for(timeout=8000)
        await signout_link.click()
    except:
        # 如果没找到就不强制退出
        print("未找到显式的 Sign Out/サインアウト 入口，跳过登出。")

async def run_one_row(browser, row: dict, slow_mo_ms: int = 80):
    apple_id = str(row.get("apple-email", "")).strip()
    password = str(row.get("pass", "")).strip()
    ordernumber = str(row.get("ordernumber", "")).strip() if row.get("ordernumber") is not None else ""

    newname = str(row.get("newname", "")).strip()
    new_last, new_first = split_name_jp(newname)

    if not apple_id:
        raise ValueError("CSV 中缺少 apple（Apple ID）")

    if not password:
        # 不建议明文写入 CSV；为空则交互输入
        password = getpass(f"请输入 {apple_id} 的密码（不会回显）：")

    if not new_last:
        raise ValueError("CSV 中缺少 newname，或 newname 无法解析姓/名")

    context = await browser.new_context(
        locale="ja-JP",
        viewport={"width": 1280, "height": 840},
    )
    page = await context.new_page()

    # 让动作更“可见”
    page.set_default_timeout(30000)

    try:
        orders_page = await open_orders_page(page)
        await sign_in_if_needed(orders_page, apple_id, password)
        await open_target_order(orders_page, ordernumber or None, product_fallback="iPhone 17 Pro")
        await update_pickup_contact(orders_page, new_last, new_first)
        await sign_out(orders_page)

        print(f"[OK] {apple_id} / order={ordernumber or '(fallback by product name)'} / new={new_last} {new_first}")
    except Exception as e:
        # 失败时截图，便于你定位页面变化
        os.makedirs("pw_debug", exist_ok=True)
        shot = f"pw_debug/fail_{re.sub(r'[^a-zA-Z0-9]+','_', apple_id)[:30]}.png"
        try:
            await page.screenshot(path=shot, full_page=True)
            print(f"[FAIL] 已截图：{shot}")
        except:
            pass
        raise
    finally:
        await context.close()

async def main(csv_path=CSV_PATH, headless=False):
    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, slow_mo=80)
        try:
            for row in rows:
                await run_one_row(browser, row)
        finally:
            await browser.close()

# 在 Jupyter 里直接运行：
# await main(CSV_PATH, headless=False)


In [ ]:
# 改过
import asyncio
import json
import os
import re
from pathlib import Path
from getpass import getpass
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright, Browser, BrowserContext, Page

# ============ 配置常量 ============
AUTH_CACHE_FILE = ".auth_cache.json"  # 保存会话信息
DEBUG_DIR = "pw_debug"

# ============ Apple Store 认证信息 ============
APPLE_STORE_COOKIES = {
    "pxro": "1",              # PerimeterX 反爬虫标识
    "geo": "JP",              # 地理位置
    "s_cc": "true",           # Cookie 同意
    "at_check": "true",       # Adblock 检测
}

APPLE_STORE_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "ja-JP,ja;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.apple.com/jp/",
}

# ============ 工具函数 ============

def split_name_jp(full_name: str) -> tuple[str, str]:
    """分割日语名字为 (姓, 名)"""
    parts = full_name.strip().split()
    if len(parts) >= 2:
        return parts[0], " ".join(parts[1:])
    elif len(parts) == 1:
        return parts[0], ""
    return "", ""


async def load_auth_cache() -> dict:
    """从缓存文件加载认证信息"""
    if Path(AUTH_CACHE_FILE).exists():
        try:
            with open(AUTH_CACHE_FILE, "r") as f:
                cache = json.load(f)
                # 检查缓存是否过期（超过 6 小时）
                cache_time = cache.get("timestamp", 0)
                if datetime.now().timestamp() - cache_time < 6 * 3600:
                    print(f"✅ 使用缓存的认证信息")
                    return cache
                else:
                    print(f"⚠️  认证缓存已过期")
                    return {}
        except Exception as e:
            print(f"⚠️  读取缓存失败: {e}")
            return {}
    return {}


async def save_auth_cache(auth_data: dict):
    """保存认证信息到缓存"""
    try:
        auth_data["timestamp"] = datetime.now().timestamp()
        with open(AUTH_CACHE_FILE, "w") as f:
            json.dump(auth_data, f, indent=2)
        print(f"💾 认证信息已保存到缓存")
    except Exception as e:
        print(f"⚠️  保存缓存失败: {e}")


async def inject_apple_cookies(context: BrowserContext):
    """注入 Apple Store 的必要 cookies"""
    try:
        await context.add_cookies([
            {
                "name": name,
                "value": value,
                "domain": ".store.apple.com",
                "path": "/",
                "expires": int(datetime.now().timestamp()) + 86400,  # 24 小时过期
            }
            for name, value in APPLE_STORE_COOKIES.items()
        ])
        print("✅ Apple Store cookies 已注入")
    except Exception as e:
        print(f"⚠️  注入 cookies 失败: {e}")


async def open_orders_page(page: Page) -> Page:
    """打开订单查询页面"""
    print("🔄 打开订单查询页面...")

    try:
        await page.goto(
            "https://secure8.store.apple.com/jp/shop/signIn/orders",
            wait_until="networkidle",
            timeout=30000
        )
        print(f"✅ 订单页面已加载: {page.url}")
        return page
    except Exception as e:
        print(f"❌ 打开页面失败: {e}")
        raise

async def check_if_logged_in(page: Page) -> bool:
    """检查是否已登录"""

    # 最可靠的方法：检查 URL
    if "/order/" in page.url and "/signIn/" not in page.url:
        return True

    # 备选方法：检查密码字段
    password_field = await page.query_selector('input[type="password"]')
    if password_field and await password_field.is_visible():
        return False

    # 默认为已登录
    return True

async def sign_in_if_needed(page: Page, apple_id: str, password: str, slow_mo_ms: int = 80):
    """如果需要则进行登录"""
    print(f"🔐 检查登录状态...")


    try:
        # 等待登录表单或已登录标记
        # login_form = await page.query_selector('input[type="password"]')
        is_logged_in = await check_if_logged_in(page)
        # already_signed = await page.query_selector('text=ご注文')

        if is_logged_in:
            print("✅ 已经登录，跳过登录流程")
            return

        # if not login_form:
        #     print("⚠️  无法找到登录表单，等待 3 秒后重试...")
        #     await page.wait_for_timeout(3000)

        # 填充登录信息
        print(f"📝 填充 Apple ID: {apple_id}")
        await page.fill('input[type="email"], input[placeholder*="メール"]', apple_id, delay=50)

        await page.wait_for_timeout(int(slow_mo_ms))

        # 填充密码
        print(f"🔑 填充密码...")
        await page.fill('input[type="password"]', password, delay=50)

        await page.wait_for_timeout(int(slow_mo_ms))

        # 点击登录按钮
        print(f"🚀 点击登录按钮...")
        submit_button = await page.query_selector('button[type="submit"]')
        if submit_button:
            await submit_button.click()
        else:
            await page.press('input[type="password"]', 'Enter')

        # 等待登录完成（可能有多步验证）
        await page.wait_for_url("**/order/**", timeout=30000)

        # 保存会话 cookies 到缓存
        cookies = await page.context.cookies()
        await save_auth_cache({
            "cookies": [
                {
                    "name": c["name"],
                    "value": c["value"],
                    "domain": c.get("domain"),
                    "path": c.get("path"),
                }
                for c in cookies
            ],
            "apple_id": apple_id,
        })

        print(f"✅ 登录成功！")

    except Exception as e:
        print(f"❌ 登录失败: {e}")
        raise


async def open_target_order(page: Page, ordernumber: str | None = None, product_fallback: str = "iPhone 17 Pro"):
    """打开目标订单"""
    print(f"📦 查找订单...")

    try:
        if ordernumber:
            print(f"🔍 搜索订单号: {ordernumber}")
            # 填充订单号搜索
            order_input = await page.query_selector('input[placeholder*="注文番号"]')
            if order_input:
                await order_input.fill(ordernumber, delay=50)
                await page.press('input[placeholder*="注文番号"]', 'Enter')
                await page.wait_for_url("**/order/**", timeout=15000)
        else:
            print(f"📱 使用产品名称回退: {product_fallback}")
            # 按产品名称搜索（在列表中找到）
            await page.wait_for_timeout(2000)
            product_selector = f'text="{product_fallback}"'
            try:
                await page.click(product_selector)
            except:
                print(f"⚠️  未找到 {product_fallback}，使用第一个订单")
                first_order = await page.query_selector('a[href*="/order/"]')
                if first_order:
                    await first_order.click()

        await page.wait_for_url("**/order/**", timeout=15000)
        print(f"✅ 订单页面已打开")

    except Exception as e:
        print(f"❌ 打开订单失败: {e}")
        raise


async def update_pickup_contact(page: Page, last_name: str, first_name: str):
    """更新取货联系人信息"""
    print(f"👤 更新联系人信息: {last_name} {first_name}")

    try:
        # 查找编辑按钮
        edit_button = await page.query_selector('button:has-text("編集")')
        if edit_button:
            await edit_button.click()
            await page.wait_for_timeout(1500)

        # 更新姓氏
        last_name_input = await page.query_selector('input[placeholder*="姓"]')
        if last_name_input:
            await last_name_input.fill("")
            await last_name_input.fill(last_name, delay=30)
            print(f"  ✓ 姓: {last_name}")

        await page.wait_for_timeout(500)

        # 更新名字
        first_name_input = await page.query_selector('input[placeholder*="名"]')
        if first_name_input:
            await first_name_input.fill("")
            await first_name_input.fill(first_name, delay=30)
            print(f"  ✓ 名: {first_name}")

        # 保存更改
        save_button = await page.query_selector('button:has-text("保存")')
        if save_button:
            await save_button.click()
            await page.wait_for_timeout(2000)
            print(f"✅ 联系人信息已更新")

    except Exception as e:
        print(f"❌ 更新联系人失败: {e}")
        raise


async def sign_out(page: Page):
    """登出"""
    print(f"🚪 正在登出...")

    try:
        # 打开账户菜单
        account_menu = await page.query_selector('button[aria-label*="Account"]')
        if account_menu:
            await account_menu.click()
            await page.wait_for_timeout(500)

        # 点击登出
        signout_button = await page.query_selector('button:has-text("サインアウト")')
        if signout_button:
            await signout_button.click()
            await page.wait_for_url("**/signIn/**", timeout=10000)
            print(f"✅ 已登出")

    except Exception as e:
        print(f"⚠️  登出失败（非关键）: {e}")


# ============ 主流程 ============

async def run_one_row(browser: Browser, row: dict, slow_mo_ms: int = 80):
    """处理单行数据"""

    apple_id = str(row.get("apple-email", "")).strip()
    password = str(row.get("pass", "")).strip()
    ordernumber = str(row.get("ordernumber", "")).strip() if row.get("ordernumber") is not None else ""
    newname = str(row.get("newname", "")).strip()

    new_last, new_first = split_name_jp(newname)

    # 参数验证
    if not apple_id:
        raise ValueError("CSV 中缺少 apple-email")

    if not password:
        password = getpass(f"请输入 {apple_id} 的密码（不会回显）：")

    if not new_last:
        raise ValueError("CSV 中缺少 newname，或 newname 无法解析姓/名")

    # 创建浏览器上下文
    context = await browser.new_context(
        locale="ja-JP",
        viewport={"width": 1280, "height": 840},
        extra_http_headers=APPLE_STORE_HEADERS,
    )

    page = await context.new_page()
    page.set_default_timeout(30000)

    # 注入 cookies
    await inject_apple_cookies(context)

    # 尝试使用缓存的认证信息
    auth_cache = await load_auth_cache()
    if auth_cache.get("cookies"):
        try:
            await context.add_cookies(auth_cache["cookies"])
            print("✅ 使用缓存的会话 cookies")
        except:
            pass

    try:
        print(f"\\n{'='*60}")
        print(f"👤 处理用户: {apple_id}")
        print(f"{'='*60}")

        orders_page = await open_orders_page(page)
        await sign_in_if_needed(orders_page, apple_id, password, slow_mo_ms)
        await open_target_order(orders_page, ordernumber or None, product_fallback="iPhone 17 Pro")
        await update_pickup_contact(orders_page, new_last, new_first)
        await sign_out(orders_page)

        print(f"✅ [OK] {apple_id} / order={ordernumber or '(fallback)'} / name={new_last} {new_first}")

    except Exception as e:
        # 失败时截图
        os.makedirs(DEBUG_DIR, exist_ok=True)
        shot = f"{DEBUG_DIR}/fail_{re.sub(r'[^a-zA-Z0-9]+', '_', apple_id)[:30]}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"

        try:
            await page.screenshot(path=shot, full_page=True)
            print(f"📸 调试截图已保存: {shot}")
        except:
            pass

        print(f"❌ [FAIL] {apple_id} - {str(e)}")
        raise

    finally:
        await context.close()


async def main(csv_path: str = CSV_PATH, headless: bool = False):
    """主程序"""

    print(f"📄 读取 CSV: {csv_path}")
    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")

    print(f"📊 总共需要处理 {len(rows)} 行")
    print(f"{'='*60}\\n")

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=headless,
            slow_mo=80,
            args=[
                "--disable-blink-features=AutomationControlled",  # 隐藏自动化标记
                "--disable-dev-shm-usage",
            ]
        )

        success_count = 0
        fail_count = 0

        try:
            for idx, row in enumerate(rows, 1):
                print(f"\\n[进度] {idx}/{len(rows)}")

                try:
                    await run_one_row(browser, row)
                    success_count += 1

                    # 每条记录之间稍作休息，避免被检测
                    if idx < len(rows):
                        await asyncio.sleep(2)

                except Exception as e:
                    fail_count += 1
                    print(f"⚠️  记录 #{idx} 失败，继续下一条...")
                    await asyncio.sleep(3)

        finally:
            await browser.close()

            print(f"\\n{'='*60}")
            print(f"📊 处理完成！")
            print(f"✅ 成功: {success_count}")
            print(f"❌ 失败: {fail_count}")
            print(f"{'='*60}")


# ============ 入口 ============
await main(CSV_PATH, headless=False)
#
# if __name__ == "__main__":
#     # 运行模式选择
#     import sys
#
#     headless_mode = "--headless" in sys.argv
#     csv_file = None
#
#     for arg in sys.argv[1:]:
#         if arg.endswith(".csv"):
#             csv_file = arg
#
#     asyncio.run(main(csv_file or CSV_PATH, headless=headless_mode))

📄 读取 CSV: ../some-data/Book10.csv
📊 总共需要处理 1 行
============================================================\n
\n[进度] 1/1
✅ Apple Store cookies 已注入
\n============================================================
👤 处理用户: j5x648@icloud.com
🔄 打开订单查询页面...
✅ 订单页面已加载: https://www.apple.com/jp/shop/sorry/illegal_arguments
🔐 检查登录状态...
✅ 已经登录，跳过登录流程
📦 查找订单...
🔍 搜索订单号: W1610977865


In [ ]:
async def main(csv_path=CSV_PATH, headless=False):
    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, slow_mo=80)
        try:
            for row in rows:
                await run_one_row(browser, row)
        finally:
            await browser.close()

# 在 Jupyter 里直接运行：
await main(CSV_PATH, headless=False)

In [18]:
import asyncio
import json
import os
import re
import time
from pathlib import Path
from getpass import getpass
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright, Browser, BrowserContext, Page

# ============ 配置常量（可根据需要修改）============
APPLE_HOME = "https://www.apple.com/jp/"
CSV_PATH = "../some-data/Book10.csv"  # 修改为你的 CSV 路径
AUTH_CACHE_FILE = ".auth_cache.json"
DEBUG_DIR = "pw_debug"

# ============ UI.Vision 固定元素 ID ============
BTN_UPDATE_PICKUP = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact"
INP_LASTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName"
INP_FIRSTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName"
BTN_SUBMIT = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit"

print("✅ 配置已加载")

✅ 配置已加载


In [19]:
# ============ 认证缓存管理 ============

async def load_auth_cache(apple_id: str) -> dict | None:
    """加载特定 Apple ID 的认证缓存"""
    if not Path(AUTH_CACHE_FILE).exists():
        return None

    try:
        with open(AUTH_CACHE_FILE, "r") as f:
            cache = json.load(f)

        if cache.get("apple_id") == apple_id:
            cache_time = cache.get("timestamp", 0)
            if datetime.now().timestamp() - cache_time < 12 * 3600:
                print(f"✅ 使用缓存的认证信息（Apple ID: {apple_id}）")
                return cache
            else:
                print(f"⚠️  认证缓存已过期")
                return None
    except Exception as e:
        print(f"⚠️  读取缓存失败: {e}")
        return None

    return None


async def save_auth_cache(apple_id: str, cookies: list):
    """保存认证信息到缓存"""
    try:
        cache_data = {
            "apple_id": apple_id,
            "cookies": [
                {
                    "name": c["name"],
                    "value": c["value"],
                    "domain": c.get("domain"),
                    "path": c.get("path"),
                    "secure": c.get("secure"),
                    "httpOnly": c.get("httpOnly"),
                }
                for c in cookies if c.get("name")
            ],
            "timestamp": datetime.now().timestamp(),
        }

        with open(AUTH_CACHE_FILE, "w") as f:
            json.dump(cache_data, f, indent=2)

        print(f"💾 认证信息已保存（Apple ID: {apple_id}）")
    except Exception as e:
        print(f"⚠️  保存缓存失败: {e}")


# ============ 工具函数 ============

def split_name_jp(fullname: str):
    """将 '神原　龍一' / '神原 龍一' 这类名字拆成 (last, first)"""
    if not fullname or not isinstance(fullname, str):
        return "", ""

    s = fullname.replace("　", " ").strip()
    parts = [p for p in re.split(r"\\s+", s) if p]

    if len(parts) >= 2:
        return parts[0], parts[1]
    return parts[0], ""


async def maybe_switch_to_popup_or_same_page(context, current_page, click_coro, wait_ms=1200):
    """检查是否打开新页面"""
    before = len(context.pages)
    await click_coro
    await current_page.wait_for_timeout(wait_ms)
    after = len(context.pages)

    return context.pages[-1] if after > before else current_page

print("✅ 辅助函数已加载")

✅ 辅助函数已加载


In [23]:
# ============ 核心页面流程函数 ============

async def open_orders_page(page: Page, context: BrowserContext) -> Page:
    """从 Apple 首页导航到订单页面"""
    print("🔄 打开订单页面...")

    try:
        await page.goto(APPLE_HOME, wait_until="domcontentloaded", timeout=30000)
        print(f"✅ 已加载首页")

        await page.locator("#globalnav-menubutton-link-bag").wait_for(timeout=10000)
        await page.locator("#globalnav-menubutton-link-bag").click()

        bag_menu = page.locator("#globalnav-submenu-bag")
        orders_link = bag_menu.get_by_role(
            "link",
            name=re.compile(r"(ご注文|注文|Orders|注文状況|ご注文状況)")
        ).first

        await orders_link.wait_for(timeout=15000)
        print(f"✅ 找到订单链接")

        orders_page = await maybe_switch_to_popup_or_same_page(
            context, page, orders_link.click(), wait_ms=1500
        )

        await orders_page.wait_for_load_state("domcontentloaded", timeout=30000)
        print(f"✅ 订单页已打开: {orders_page.url}")

        return orders_page

    except Exception as e:
        print(f"❌ 打开订单页失败: {e}")
        raise


async def sign_in_if_needed(page: Page, apple_id: str, password: str):
    """改进的登录检测和执行逻辑"""
    print(f"🔐 检查登录状态（Apple ID: {apple_id}）...")

    await page.wait_for_load_state("domcontentloaded", timeout=30000)

    # 步骤 1: 尝试使用缓存
    # auth_cache = await load_auth_cache(apple_id)
    # if auth_cache and auth_cache.get("cookies"):
    #     try:
    #         await page.context.add_cookies(auth_cache["cookies"])
    #         print(f"✅ 已注入缓存的 cookies，正在验证...")
    #
    #         await page.reload(wait_until="domcontentloaded", timeout=30000)
    #
    #         frame_loc = page.frame_locator("iframe").first
    #         account_field = frame_loc.locator("#account_name_text_field")
    #
    #         try:
    #             await account_field.wait_for(timeout=3000)
    #             print(f"⚠️  缓存的认证信息已失效，需要重新登录")
    #         except:
    #             print(f"✅ 缓存认证有效，已登录")
    #             return
    #
    #     except Exception as e:
    #         print(f"⚠️  使用缓存失败: {e}")

    # 步骤 2: 检测 iframe 登录框
    frame_loc = page.frame_locator("iframe").first
    account_field = frame_loc.locator("#account_name_text_field")

    try:
        await account_field.wait_for(timeout=6000)
        print(f"📝 检测到登录框，正在执行登录...")
    except:
        print(f"✅ 未检测到登录框，认为已登录")
        return

    # 步骤 3: 执行登录
    try:
        print(f"📝 填充 Apple ID...")
        await account_field.fill(apple_id)

        sign_in_btn = frame_loc.locator("#sign-in")
        await sign_in_btn.click()

        time.sleep(10)

        print(f"🔑 等待密码字段...")
        password_field = frame_loc.locator("#password_text_field")
        await password_field.wait_for(timeout=15000)

        print(f"🔑 填充密码...")
        await password_field.fill(password)

        time.sleep(10)

        await sign_in_btn.click()

        print(f"⏳ 等待登录完成（包括 2FA 验证）...")

        # 步骤 4: 等待登录完成
        try:
            await page.wait_for_selector("#rs-container", timeout=180000)
            print(f"✅ 检测到订单列表，登录成功")
        except:
            print(f"⏳ 未自动检测到订单列表")
            print(f"💡 如果出现 2FA 验证，请在浏览器中完成...")

            try:
                await page.wait_for_selector("#rs-container", timeout=120000)
                print(f"✅ 登录成功！")
            except:
                input("❓ 请确认已完成登录和 2FA，然后按回车继续：")

        # 步骤 5: 保存认证信息
        cookies = await page.context.cookies()
        await save_auth_cache(apple_id, cookies)

    except Exception as e:
        print(f"❌ 登录过程失败: {e}")
        raise


async def open_target_order(page: Page, ordernumber: str | None = None, product_fallback: str = "iPhone 17 Pro"):
    """打开目标订单"""
    print(f"📦 打开订单...")

    try:
        await page.wait_for_load_state("domcontentloaded", timeout=30000)

        if ordernumber and isinstance(ordernumber, str) and ordernumber.strip():
            on = ordernumber.strip()
            print(f"🔍 搜索订单号: {on}")

            try:
                link = page.get_by_role("link", name=re.compile(re.escape(on))).first
                await link.wait_for(timeout=10000)
                await link.click()
                print(f"✅ 通过链接打开订单")
                await page.wait_for_load_state("domcontentloaded", timeout=30000)
                return
            except:
                pass

            try:
                txt = page.get_by_text(on).first
                await txt.wait_for(timeout=10000)
                await txt.click()
                print(f"✅ 通过文本打开订单")
                await page.wait_for_load_state("domcontentloaded", timeout=30000)
                return
            except:
                pass

            print(f"⚠️  未能通过订单号打开，回退到商品名")

        print(f"📱 使用商品名回退: {product_fallback}")
        prod_link = page.get_by_role("link", name=re.compile(re.escape(product_fallback))).first
        await prod_link.wait_for(timeout=15000)
        await prod_link.click()

        await page.wait_for_load_state("domcontentloaded", timeout=30000)
        print(f"✅ 已打开订单详情页")

    except Exception as e:
        print(f"❌ 打开订单失败: {e}")
        raise


async def update_pickup_contact(page: Page, new_last: str, new_first: str):
    """更新取货联系人 - 改进版"""
    print(f"👤 更新取货联系人: {new_last} {new_first}")

    try:
        # 确保页面完全加载
        await page.wait_for_load_state("networkidle", timeout=30000)

        # 额外等待时间，确保 JS 渲染完成
        await page.wait_for_timeout(2000)

        # ========== 方案 A: 使用 ID 定位器（推荐）==========
        update_btn = page.locator(f"#{BTN_UPDATE_PICKUP}")

        # 等待按钮出现和可见
        await update_btn.wait_for(state="visible", timeout=20000)

        print(f"✅ 找到更新按钮")

        # 尝试方案 1: 使用 click()
        try:
            await update_btn.click()
            print(f"✓ 已点击更新按钮（方案 1）")
        except:
            # 备选方案 2: 使用 JavaScript 点击
            print(f"⚠️  普通点击失败，尝试 JS 点击...")
            await page.evaluate(f"document.getElementById('{BTN_UPDATE_PICKUP}').click()")
            print(f"✓ 已通过 JS 点击更新按钮（方案 2）")

        # 等待对话框或编辑区域出现
        await page.wait_for_timeout(1500)

        # ========== 填充姓 ==========
        last_name_input = page.locator(f"#{INP_LASTNAME}")

        # 使用 visible 状态等待
        try:
            await last_name_input.wait_for(state="visible", timeout=15000)
        except:
            # 如果找不到，尝试等待任何状态的元素
            await last_name_input.wait_for(timeout=15000)

        print(f"  ✓ 找到姓输入框")

        # 清空并填充
        await last_name_input.clear()
        await last_name_input.fill(new_last, delay=30)
        print(f"  ✓ 填充姓: {new_last}")

        await page.wait_for_timeout(500)

        # ========== 填充名 ==========
        first_name_input = page.locator(f"#{INP_FIRSTNAME}")

        try:
            await first_name_input.wait_for(state="visible", timeout=15000)
        except:
            await first_name_input.wait_for(timeout=15000)

        print(f"  ✓ 找到名输入框")

        await first_name_input.clear()
        await first_name_input.fill(new_first, delay=30)
        print(f"  ✓ 填充名: {new_first}")

        await page.wait_for_timeout(500)

        # ========== 提交 ==========
        submit_btn = page.locator(f"#{BTN_SUBMIT}")

        try:
            await submit_btn.wait_for(state="visible", timeout=15000)
        except:
            await submit_btn.wait_for(timeout=15000)

        print(f"✓ 找到提交按钮")

        # 点击提交
        try:
            await submit_btn.click()
            print(f"✓ 已提交（方案 1）")
        except:
            print(f"⚠️  普通提交失败，尝试 JS 提交...")
            await page.evaluate(f"document.getElementById('{BTN_SUBMIT}').click()")
            print(f"✓ 已通过 JS 提交（方案 2）")

        # 等待提交完成
        await page.wait_for_timeout(2500)

        print(f"✅ 联系人信息已更新")

    except Exception as e:
        print(f"❌ 更新联系人失败: {e}")

        # 出错时的调试信息
        print(f"\\n🔍 调试信息：")
        print(f"  页面 URL: {page.url}")

        # 尝试找到按钮以验证存在性
        try:
            btn_exists = await page.locator(f"#{BTN_UPDATE_PICKLE}").is_visible()
            print(f"  更新按钮可见: {btn_exists}")
        except:
            print(f"  无法检查更新按钮状态")

        raise

async def sign_out(page: Page):
    """执行登出"""
    print(f"🚪 执行登出...")

    try:
        bag_btn = page.locator("#globalnav-menubutton-link-bag")
        await bag_btn.wait_for(timeout=10000)
        await bag_btn.click()

        bag_menu = page.locator("#globalnav-submenu-bag")
        signout_link = bag_menu.get_by_role(
            "link",
            name=re.compile(r"(サインアウト|Sign Out|ログアウト)")
        ).first

        try:
            await signout_link.wait_for(timeout=8000)
            await signout_link.click()
            print(f"✅ 已登出")
        except:
            print(f"⚠️  未找到显式的登出入口（非关键），继续")

    except Exception as e:
        print(f"⚠️  登出失败（非关键）: {e}")

print("✅ 核心函数已加载")

✅ 核心函数已加载


In [24]:
# ============ 主流程函数 ============

async def run_one_row(browser: Browser, row: dict):
    """处理单行数据"""

    apple_id = str(row.get("apple-email", "")).strip()
    password = str(row.get("pass", "")).strip()
    ordernumber = str(row.get("ordernumber", "")).strip() if row.get("ordernumber") is not None else ""
    newname = str(row.get("newname", "")).strip()

    new_last, new_first = split_name_jp(newname)

    # 参数验证
    if not apple_id:
        raise ValueError("CSV 中缺少 apple-email")

    if not password:
        password = getpass(f"请输入 {apple_id} 的密码（不会回显）：")

    if not new_last:
        raise ValueError("CSV 中缺少 newname，或 newname 无法解析姓/名")

    # 创建浏览器上下文
    context = await browser.new_context(
        locale="ja-JP",
        viewport={"width": 1280, "height": 840},
    )

    page = await context.new_page()
    page.set_default_timeout(30000)

    try:
        print(f"\\n{'='*70}")
        print(f"👤 处理用户: {apple_id}")
        print(f"📦 订单号: {ordernumber or '(使用商品名回退)'}")
        print(f"👤 新名字: {new_last} {new_first}")
        print(f"{'='*70}")

        # 主流程
        orders_page = await open_orders_page(page, context)
        await sign_in_if_needed(orders_page, apple_id, password)
        await open_target_order(orders_page, ordernumber or None, product_fallback="iPhone 17 Pro")
        await update_pickup_contact(orders_page, new_last, new_first)
        await sign_out(orders_page)

        print(f"\\n✅ [OK] {apple_id} / order={ordernumber or '(fallback)'} / name={new_last} {new_first}\\n")
        return True

    except Exception as e:
        # 失败时截图
        os.makedirs(DEBUG_DIR, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        shot = f"{DEBUG_DIR}/fail_{re.sub(r'[^a-zA-Z0-9]+', '_', apple_id)[:30]}_{timestamp}.png"

        try:
            await page.screenshot(path=shot, full_page=True)
            print(f"📸 调试截图: {shot}")
        except:
            pass

        print(f"\\n❌ [FAIL] {apple_id}")
        print(f"   错误: {str(e)}\\n")
        return False

    finally:
        await context.close()


async def run_batch(csv_path: str, headless: bool = True):
    """
    Jupyter 友好的批处理函数
    """

    print(f"\\n{'='*70}")
    print(f"📋 读取 CSV: {csv_path}")

    # 检查文件存在
    if not Path(csv_path).exists():
        print(f"❌ 文件不存在: {csv_path}")
        return

    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")

    print(f"📊 总共需要处理 {len(rows)} 行")
    print(f"{'='*70}\\n")

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=headless,
            slow_mo=80,
            args=[
                "--disable-blink-features=AutomationControlled",
                "--disable-dev-shm-usage",
            ]
        )

        success_count = 0
        fail_count = 0
        results = []

        try:
            for idx, row in enumerate(rows, 1):
                try:
                    print(f"[进度 {idx}/{len(rows)}]")
                    success = await run_one_row(browser, row)

                    if success:
                        success_count += 1
                        results.append({
                            "apple_id": row.get("apple-email"),
                            "status": "✅ OK",
                            "order": row.get("ordernumber", "(fallback)"),
                            "name": row.get("newname")
                        })
                    else:
                        fail_count += 1
                        results.append({
                            "apple_id": row.get("apple-email"),
                            "status": "❌ FAIL",
                            "order": row.get("ordernumber", "(fallback)"),
                            "name": row.get("newname")
                        })

                    # 每条记录之间稍作休息
                    if idx < len(rows):
                        await asyncio.sleep(2)

                except Exception as e:
                    fail_count += 1
                    results.append({
                        "apple_id": row.get("apple-email"),
                        "status": "❌ ERROR",
                        "order": row.get("ordernumber", "(fallback)"),
                        "name": row.get("newname"),
                        "error": str(e)
                    })
                    print(f"⚠️  记录 #{idx} 失败，继续下一条...")
                    await asyncio.sleep(3)

        finally:
            await browser.close()

            # 输出结果统计
            print(f"\\n{'='*70}")
            print(f"📊 处理完成！")
            print(f"✅ 成功: {success_count}/{len(rows)}")
            print(f"❌ 失败: {fail_count}/{len(rows)}")
            print(f"{'='*70}\\n")

            # 显示结果表格
            results_df = pd.DataFrame(results)
            print(results_df.to_string(index=False))

            return {
                "total": len(rows),
                "success": success_count,
                "fail": fail_count,
                "results": results_df
            }

print("✅ 主函数已加载")

✅ 主函数已加载


In [25]:
# ============ 执行入口（Jupyter 友好版）============

# 【可选】先确认 CSV 文件
csv_file = CSV_PATH
print(f"CSV 路径: {csv_file}")
print(f"文件是否存在: {Path(csv_file).exists()}")

# 如果需要修改，取消注释并修改：
# csv_file = "../some-data/Book10.csv"  # 改为你的实际路径

if Path(csv_file).exists():
    print("\\n开始运行...")
    # Jupyter 中运行异步代码的标准方式
    result = await run_batch(csv_file, headless=False)
    print("\\n✅ 所有任务完成！")
else:
    print(f"\\n❌ CSV 文件不存在，请检查路径：{csv_file}")

CSV 路径: ../some-data/Book10.csv
文件是否存在: True
\n开始运行...
\n======================================================================
📋 读取 CSV: ../some-data/Book10.csv
📊 总共需要处理 1 行
======================================================================\n
[进度 1/1]
\n======================================================================
👤 处理用户: j5x648@icloud.com
📦 订单号: W1610977865
👤 新名字: 神原 龍一 
🔄 打开订单页面...
✅ 已加载首页
✅ 找到订单链接
✅ 订单页已打开: https://secure6.store.apple.com/jp/shop/signIn/orders?ssi=4AAABnAKH3LQBIIAN68kzOwebkl-B6G4Clze9sJlU4LE4ONLnsqosg66QAAAANWh0dHBzOi8vc2VjdXJlNi5zdG9yZS5hcHBsZS5jb20vanAvc2hvcC9vcmRlci9saXN0fHx8AAIBfqq-cFBfHwfTvZ4fRWX0wDXFa_wO35GzXCtREZ0XbsA
🔐 检查登录状态（Apple ID: j5x648@icloud.com）...
📝 检测到登录框，正在执行登录...
📝 填充 Apple ID...
🔑 等待密码字段...
🔑 填充密码...
⏳ 等待登录完成（包括 2FA 验证）...
✅ 检测到订单列表，登录成功
💾 认证信息已保存（Apple ID: j5x648@icloud.com）
📦 打开订单...
🔍 搜索订单号: W1610977865
✅ 通过链接打开订单
👤 更新取货联系人: 神原 龍一 
❌ 更新联系人失败: Locator.wait_for: Timeout 20000ms exceeded.
Call log:
  - waiting for locator("#orderD

In [1]:
# ============ 完整的单个单元格版本 ============
# 在 Jupyter 中直接运行这个单元格

import asyncio
import json
import os
import re
import time
from pathlib import Path
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright

# ============ 配置常量（全部在这里） ============
APPLE_HOME = "https://www.apple.com/jp/"
CSV_PATH = "../some-data/Book10.csv"  # 修改为你的实际路径
AUTH_CACHE_FILE = ".auth_cache.json"
DEBUG_DIR = "pw_debug"

# ============ UI.Vision 元素 ID（检查这些是否正确） ============
BTN_UPDATE_PICKUP = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact"
INP_LASTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName"
INP_FIRSTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName"
BTN_SUBMIT = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit"

print("✅ 常量定义完成")
print(f"  编辑按钮 ID: {BTN_UPDATE_PICKUP}")
print(f"  姓输入框 ID: {INP_LASTNAME}")
print(f"  名输入框 ID: {INP_FIRSTNAME}")
print(f"  提交按钮 ID: {BTN_SUBMIT}")

# ============ 工具函数 ============

def split_name_jp(fullname: str):
    """将 '神原　龍一' / '神原 龍一' 拆成 (last, first)"""
    if not fullname or not isinstance(fullname, str):
        return "", ""

    s = fullname.replace("　", " ").strip()
    parts = [p for p in re.split(r"\\s+", s) if p]

    if len(parts) >= 2:
        return parts[0], parts[1]
    return parts[0], ""


async def load_auth_cache(apple_id: str) -> dict | None:
    """加载认证缓存"""
    if not Path(AUTH_CACHE_FILE).exists():
        return None

    try:
        with open(AUTH_CACHE_FILE, "r") as f:
            cache = json.load(f)

        if cache.get("apple_id") == apple_id:
            cache_time = cache.get("timestamp", 0)
            if datetime.now().timestamp() - cache_time < 12 * 3600:
                print(f"✅ 使用缓存的认证信息")
                return cache
    except:
        pass

    return None


async def save_auth_cache(apple_id: str, cookies: list):
    """保存认证缓存"""
    try:
        cache_data = {
            "apple_id": apple_id,
            "cookies": [
                {
                    "name": c["name"],
                    "value": c["value"],
                    "domain": c.get("domain"),
                    "path": c.get("path"),
                }
                for c in cookies if c.get("name")
            ],
            "timestamp": datetime.now().timestamp(),
        }

        with open(AUTH_CACHE_FILE, "w") as f:
            json.dump(cache_data, f, indent=2)

        print(f"💾 认证信息已保存")
    except Exception as e:
        print(f"⚠️  保存缓存失败: {e}")


async def maybe_switch_to_popup_or_same_page(context, current_page, click_coro, wait_ms=1200):
    """检查是否打开新页面"""
    before = len(context.pages)
    await click_coro
    await current_page.wait_for_timeout(wait_ms)
    after = len(context.pages)

    return context.pages[-1] if after > before else current_page


# ============ 核心函数 ============

async def open_orders_page(page, context):
    """打开订单页面"""
    print("🔄 打开订单页面...")

    await page.goto(APPLE_HOME, wait_until="domcontentloaded", timeout=30000)
    await page.locator("#globalnav-menubutton-link-bag").wait_for(timeout=10000)
    await page.locator("#globalnav-menubutton-link-bag").click()

    bag_menu = page.locator("#globalnav-submenu-bag")
    orders_link = bag_menu.get_by_role(
        "link",
        name=re.compile(r"(ご注文|注文|Orders|注文状況|ご注文状況)")
    ).first

    await orders_link.wait_for(timeout=15000)

    orders_page = await maybe_switch_to_popup_or_same_page(
        context, page, orders_link.click(), wait_ms=1500
    )

    await orders_page.wait_for_load_state("domcontentloaded", timeout=30000)
    print(f"✅ 订单页已打开")

    return orders_page


async def sign_in_if_needed(page, apple_id: str, password: str):
    """登录流程"""
    print(f"🔐 检查登录状态...")

    await page.wait_for_load_state("domcontentloaded", timeout=30000)

    # 尝试使用缓存
    # auth_cache = await load_auth_cache(apple_id)
    # if auth_cache and auth_cache.get("cookies"):
    #     try:
    #         await page.context.add_cookies(auth_cache["cookies"])
    #         await page.reload(wait_until="domcontentloaded", timeout=30000)
    #
    #         frame_loc = page.frame_locator("iframe").first
    #         account_field = frame_loc.locator("#account_name_text_field")
    #
    #         try:
    #             await account_field.wait_for(timeout=3000)
    #             print(f"⚠️  缓存认证已失效")
    #         except:
    #             print(f"✅ 缓存认证有效，已登录")
    #             return
    #     except:
    #         pass

    # 检查是否需要登录
    frame_loc = page.frame_locator("iframe").first
    account_field = frame_loc.locator("#account_name_text_field")

    try:
        await account_field.wait_for(timeout=6000)
        print(f"📝 需要登录")
    except:
        print(f"✅ 已登录（无登录框）")
        return

    # 执行登录
    try:
        print(f"  输入 Apple ID...")
        await account_field.fill(apple_id)
        await frame_loc.locator("#sign-in").click()

        time.sleep(1)

        print(f"  输入密码...")
        password_field = frame_loc.locator("#password_text_field")
        await password_field.wait_for(timeout=15000)
        await password_field.fill(password)

        time.sleep(1)

        await frame_loc.locator("#sign-in").click()

        print(f"⏳ 等待登录完成...")

        try:
            await page.wait_for_selector("#rs-container", timeout=180000)
            print(f"✅ 登录成功")
        except:
            print(f"⏳ 可能需要 2FA 验证...")
            try:
                await page.wait_for_selector("#rs-container", timeout=120000)
                print(f"✅ 登录成功（2FA 完成）")
            except:
                input("❓ 请确认已完成登录，按回车继续：")

        cookies = await page.context.cookies()
        await save_auth_cache(apple_id, cookies)

    except Exception as e:
        print(f"❌ 登录失败: {e}")
        raise


# async def open_target_order(page, ordernumber: str | None = None, product_fallback: str = "iPhone 17 Pro"):
#     """打开订单"""
#     print(f"📦 打开订单...")
#
#     await page.wait_for_load_state("domcontentloaded", timeout=30000)
#
#     if ordernumber and isinstance(ordernumber, str) and ordernumber.strip():
#         on = ordernumber.strip()
#         print(f"  搜索订单号: {on}")
#
#         try:
#             link = page.get_by_role("link", name=re.compile(re.escape(on))).first
#             await link.wait_for(timeout=10000)
#             await link.click()
#             await page.wait_for_load_state("domcontentloaded", timeout=30000)
#             return
#         except:
#             pass
#
#         try:
#             txt = page.get_by_text(on).first
#             await txt.wait_for(timeout=10000)
#             await txt.click()
#             await page.wait_for_load_state("domcontentloaded", timeout=30000)
#             return
#         except:
#             pass
#
#         print(f"⚠️  订单号未找到，使用商品名")
#
#     print(f"  使用商品名: {product_fallback}")
#     prod_link = page.get_by_role("link", name=re.compile(re.escape(product_fallback))).first
#     await prod_link.wait_for(timeout=15000)
#     await prod_link.click()
#
#     await page.wait_for_load_state("domcontentloaded", timeout=30000)
#     print(f"✅ 订单已打开")


# async def update_pickup_contact(page, new_last: str, new_first: str):
#     """
#     更新取货联系人 - 修复版
#
#     关键修复：
#     1. 确保所有 ID 常量正确
#     2. 使用正确的等待逻辑
#     3. 添加充分的等待时间
#     """
#     print(f"👤 更新取货联系人: {new_last} {new_first}")
#
#     try:
#         await page.wait_for_load_state("networkidle", timeout=30000)
#         await page.wait_for_timeout(2000)
#
#         # ========== 点击编辑按钮 ==========
#         print(f"  🔍 找编辑按钮...")
#
#         # 使用正确的 ID 常量
#         update_btn_id = BTN_UPDATE_PICKUP  # 确保使用了正确的常量
#
#         print(f"  使用 ID: {update_btn_id}")
#
#         update_btn = page.locator(f"#{update_btn_id}")
#         await update_btn.wait_for(state="visible", timeout=20000)
#
#         print(f"  ✅ 找到编辑按钮")
#
#         await update_btn.click()
#         print(f"  ✓ 已点击编辑按钮")
#
#         await page.wait_for_timeout(2000)
#
#         # ========== 填充姓 ==========
#         print(f"  📝 填充姓...")
#
#         last_name_id = INP_LASTNAME
#         print(f"  使用 ID: {last_name_id}")
#
#         last_name_input = page.locator(f"#{last_name_id}")
#         await last_name_input.wait_for(state="visible", timeout=15000)
#
#         print(f"  ✅ 找到姓输入框")
#
#         await last_name_input.clear()
#         await page.wait_for_timeout(300)
#         await last_name_input.fill(new_last, delay=50)
#
#         print(f"  ✓ 已填充: {new_last}")
#
#         await page.wait_for_timeout(500)
#
#         # ========== 填充名 ==========
#         print(f"  📝 填充名...")
#
#         first_name_id = INP_FIRSTNAME
#         print(f"  使用 ID: {first_name_id}")
#
#         first_name_input = page.locator(f"#{first_name_id}")
#         await first_name_input.wait_for(state="visible", timeout=15000)
#
#         print(f"  ✅ 找到名输入框")
#
#         await first_name_input.clear()
#         await page.wait_for_timeout(300)
#         await first_name_input.fill(new_first, delay=50)
#
#         print(f"  ✓ 已填充: {new_first}")
#
#         await page.wait_for_timeout(500)
#
#         # ========== 提交 ==========
#         print(f"  🚀 提交...")
#
#         submit_id = BTN_SUBMIT
#         print(f"  使用 ID: {submit_id}")
#
#         submit_btn = page.locator(f"#{submit_id}")
#         await submit_btn.wait_for(state="visible", timeout=15000)
#
#         print(f"  ✅ 找到提交按钮")
#
#         await submit_btn.click()
#         print(f"  ✓ 已提交")
#
#         await page.wait_for_timeout(3000)
#
#         print(f"✅ 联系人信息已更新\\n")
#
#     except Exception as e:
#         print(f"\\n❌ 更新失败: {e}\\n")
#
#         # 调试：输出所有使用的 ID
#         print(f"🔍 使用的 ID:")
#         print(f"  BTN_UPDATE_PICKUP: {BTN_UPDATE_PICKUP}")
#         print(f"  INP_LASTNAME: {INP_LASTNAME}")
#         print(f"  INP_FIRSTNAME: {INP_FIRSTNAME}")
#         print(f"  BTN_SUBMIT: {BTN_SUBMIT}\\n")
#
#         try:
#             await page.screenshot(path="debug_error.png")
#             print(f"📸 调试截图: debug_error.png\\n")
#         except:
#             pass
#
#         raise
#

async def open_target_order(page, ordernumber: str | None = None, product_fallback: str = "iPhone 17 Pro"):
    """打开订单"""
    print(f"📦 打开订单...")

    await page.wait_for_load_state("domcontentloaded", timeout=30000)

    # 如果提供了订单号，尝试通过订单号查找
    if ordernumber and isinstance(ordernumber, str) and ordernumber.strip():
        on = ordernumber.strip()
        print(f"  搜索订单号: {on}")

        try:
            # 尝试通过订单号找到链接
            link = page.get_by_role("link", name=re.compile(re.escape(on))).first
            await link.wait_for(timeout=10000)
            await link.click()
            await page.wait_for_load_state("domcontentloaded", timeout=30000)
            print(f"✅ 订单已打开")
            return
        except Exception as e:
            print(f"⚠️  订单号未找到: {e}，使用列表中第一个订单")

    # 如果没有订单号或订单号未找到，打开列表中第一个订单
    print(f"  打开列表中的第一个订单...")

    try:
        # 定位到订单列表容器中的第一个 li 元素
        first_order = page.locator("//*[@id='rs-container']/div/div[1]/div/ul/li[1]")
        await first_order.wait_for(timeout=15000)

        # 在第一个 li 中找到链接并点击
        first_order_link = first_order.locator("a").first
        await first_order_link.wait_for(timeout=10000)
        await first_order_link.click()

        await page.wait_for_load_state("domcontentloaded", timeout=30000)
        print(f"✅ 订单已打开")

    except Exception as e:
        print(f"❌ 无法打开第一个订单: {e}")
        raise



async def update_pickup_contact(page, new_last: str, new_first: str):
    """
    更新取货联系人 - 修复版

    从完整姓名中分离姓和名
    例如: "神原 龍一" -> 姓: "神原", 名: "龍一"
    """

    # ✅ 分离姓和名
    # name_parts = full_name.split()
    # if len(name_parts) < 2:
    #     raise ValueError(f"姓名格式错误，需要包含空格: {full_name}")
    #
    # new_last = name_parts[0]  # 姓
    # new_first = name_parts[1]  # 名
    #
    # print(f"👤 更新取货联系人: {full_name}")
    print(f"   姓: {new_last}, 名: {new_first}")

    try:
        await page.wait_for_load_state("networkidle", timeout=30000)
        await page.wait_for_timeout(2000)

        # ========== 点击编辑按钮 ==========
        print(f"  🔍 找编辑按钮...")
        update_btn = page.locator(f"//*[@id='{BTN_UPDATE_PICKUP}']")
        await update_btn.wait_for(state="visible", timeout=20000)
        print(f"  ✅ 找到编辑按钮")
        await update_btn.click()
        print(f"  ✓ 已点击编辑按钮")
        await page.wait_for_timeout(2000)

        # ========== 填充姓 ==========
        print(f"  📝 填充姓: {new_last}")
        last_name_input = page.locator(f"//*[@id='{INP_LASTNAME}']")
        await last_name_input.wait_for(state="visible", timeout=15000)
        print(f"  ✅ 找到姓输入框")
        await last_name_input.clear()
        await page.wait_for_timeout(300)
        await last_name_input.type(new_last, delay=50)
        print(f"  ✓ 已填充姓: {new_last}")
        await page.wait_for_timeout(500)

        # ========== 填充名 ==========
        print(f"  📝 填充名: {new_first}")
        first_name_input = page.locator(f"//*[@id='{INP_FIRSTNAME}']")
        await first_name_input.wait_for(state="visible", timeout=15000)
        print(f"  ✅ 找到名输入框")
        await first_name_input.clear()
        await page.wait_for_timeout(300)
        await first_name_input.type(new_first, delay=50)
        print(f"  ✓ 已填充名: {new_first}")
        await page.wait_for_timeout(500)

        # ========== 提交 ==========
        print(f"  🚀 提交...")
        submit_btn = page.locator(f"//*[@id='{BTN_SUBMIT}']")
        await submit_btn.wait_for(state="visible", timeout=15000)
        print(f"  ✅ 找到提交按钮")
        await submit_btn.click()
        print(f"  ✓ 已提交")
        await page.wait_for_timeout(3000)

        print(f"✅ 联系人信息已更新\\n")

    except Exception as e:
        print(f"\\n❌ 更新失败: {e}\\n")
        try:
            await page.screenshot(path="debug_error.png")
            print(f"📸 调试截图: debug_error.png\\n")
        except:
            pass
        raise


async def sign_out(page):
    """登出"""
    print(f"🚪 登出...")

    try:
        await page.locator("#globalnav-menubutton-link-bag").wait_for(timeout=10000)
        await page.locator("#globalnav-menubutton-link-bag").click()

        bag_menu = page.locator("#globalnav-submenu-bag")
        signout_link = bag_menu.get_by_role(
            "link",
            name=re.compile(r"(サインアウト|Sign Out|ログアウト)")
        ).first

        try:
            await signout_link.wait_for(timeout=8000)
            await signout_link.click()
            print(f"✅ 已登出\\n")
        except:
            print(f"⚠️  无登出选项（非关键）\\n")

    except Exception as e:
        print(f"⚠️  登出失败（非关键）: {e}\\n")


async def run_one_row(browser, row: dict):
    """处理一行"""

    apple_id = str(row.get("apple-email", "")).strip()
    password = str(row.get("pass", "")).strip()
    ordernumber = str(row.get("ordernumber", "")).strip() if row.get("ordernumber") is not None else ""
    newname = str(row.get("newname", "")).strip()

    new_last, new_first = split_name_jp(newname)

    # 参数验证
    if not apple_id:
        raise ValueError("缺少 apple-email 列")

    if not password:
        from getpass import getpass
        password = getpass(f"请输入 {apple_id} 的密码：")

    if not new_last:
        raise ValueError("缺少 newname 或无法解析")

    context = await browser.new_context(
        locale="ja-JP",
        viewport={"width": 1280, "height": 840},
    )

    page = await context.new_page()
    page.set_default_timeout(30000)

    try:
        print(f"\\n{'='*70}")
        print(f"👤 {apple_id} | 订单: {ordernumber or '(商品名)'} | 名字: {new_last} {new_first}")
        print(f"{'='*70}")

        orders_page = await open_orders_page(page, context)
        await sign_in_if_needed(orders_page, apple_id, password)
        await open_target_order(orders_page, ordernumber or None, product_fallback="iPhone 17 Pro")
        await update_pickup_contact(orders_page, new_last, new_first)
        await sign_out(orders_page)

        print(f"✅ [OK] {apple_id}\\n")
        return True

    except Exception as e:
        os.makedirs(DEBUG_DIR, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        shot = f"{DEBUG_DIR}/fail_{re.sub(r'[^a-zA-Z0-9]+', '_', apple_id)[:30]}_{timestamp}.png"

        try:
            await page.screenshot(path=shot, full_page=True)
            print(f"📸 {shot}\\n")
        except:
            pass

        print(f"❌ [FAIL] {apple_id}: {str(e)}\\n")
        return False

    finally:
        await context.close()


async def run_batch(csv_path: str, headless: bool = True):
    """批处理"""

    print(f"\\n{'='*70}")
    print(f"📋 CSV: {csv_path}")

    if not Path(csv_path).exists():
        print(f"❌ 文件不存在\\n")
        return

    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")

    print(f"📊 总计: {len(rows)} 行")
    print(f"{'='*70}\\n")

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=headless,
            slow_mo=80,
            args=[
                "--disable-blink-features=AutomationControlled",
                "--disable-dev-shm-usage",
            ]
        )

        success_count = 0
        fail_count = 0

        try:
            for idx, row in enumerate(rows, 1):
                try:
                    print(f"[{idx}/{len(rows)}]")
                    success = await run_one_row(browser, row)

                    if success:
                        success_count += 1
                    else:
                        fail_count += 1

                    if idx < len(rows):
                        await asyncio.sleep(2)

                except Exception as e:
                    fail_count += 1
                    print(f"❌ 错误: {str(e)}\\n")
                    await asyncio.sleep(3)

        finally:
            await browser.close()

            print(f"\\n{'='*70}")
            print(f"✅ 成功: {success_count}/{len(rows)}")
            print(f"❌ 失败: {fail_count}/{len(rows)}")
            print(f"{'='*70}\\n")


# ============ 执行 ============

print("\\n开始运行...\\n")

# 确认 CSV 路径
csv_file = CSV_PATH
print(f"CSV: {csv_file}")
print(f"存在: {Path(csv_file).exists()}\\n")

if Path(csv_file).exists():
    # 显示 CSV 前几行
    df_preview = pd.read_csv(csv_file)
    print("📋 CSV 预览:")
    print(df_preview.head())
    print()

    # 执行
    await run_batch(csv_file, headless=False)
else:
    print(f"❌ 请修改 CSV_PATH")

✅ 常量定义完成
  编辑按钮 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact
  姓输入框 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName
  名输入框 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName
  提交按钮 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit
\n开始运行...\n
CSV: ../some-data/Book10.csv
存在: True\n
📋 CSV 预览:
         apple-email      pass  ordernumber oldname newname
0  j5x648@icloud.com  Qp272727  W1610977865   高橋 光晴   神原　龍一

\n======================================================================
📋 CSV: ../some-data/Book10.csv
📊 总计: 1 行
======================================================================\n
[1/1]
\n======================================================================
👤 j5x648@icloud.com | 订单: W1610977865 | 名字: 神原 龍一 
🔄 打开订单页面...
✅ 订单页已打开
🔐 检查登录状态...
📝 需要登录
  输入 

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("#rs-container") to be visible\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("#rs-container") to be visible



❌ 登录失败: BrowserContext.cookies: Connection closed while reading from the driver
❌ [FAIL] j5x648@icloud.com: BrowserContext.cookies: Connection closed while reading from the driver\n


Exception: Browser.close: Connection closed while reading from the driver

In [20]:
from playwright.async_api import async_playwright
from playwright._impl._errors import TargetClosedError  # 用于捕获“已关闭”错误

async def run_with_trace(trace_path="trace.zip", headless=False):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless)
        context = await browser.new_context()
        tracing_started = False

        try:
            await context.tracing.start(screenshots=True, snapshots=True, sources=True)
            tracing_started = True

            page = await context.new_page()
            await page.goto("https://www.apple.com/jp/", wait_until="domcontentloaded")
            await page.wait_for_timeout(2000)

            # 你的自动化逻辑放这里...

        except Exception:
            # 这里不要 close context/browser（否则 finally 的 stop 会报你现在这个错）
            raise

        finally:
            # 先 stop tracing，再关 context/browser
            if tracing_started:
                try:
                    await context.tracing.stop(path=trace_path)
                except TargetClosedError:
                    # 说明你或代码已把窗口/上下文关了，trace 无法导出
                    print("Tracing.stop 失败：context/browser 已关闭（可能手动关闭了窗口）。")

            try:
                await context.close()
            finally:
                await browser.close()

await run_with_trace("trace.zip", headless=False)


In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def debug_541():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        async def log_bad_response(resp):
            if resp.status >= 400 or "idms/authx" in resp.url:
                try:
                    txt = await resp.text()
                except Exception:
                    txt = "<cannot read body>"
                head = txt[:800].replace("\n", "\\n")
                print(f"[RESP] {resp.status} {resp.url}\nBODY_HEAD: {head}\n")
                # 保存完整响应（可能很长）
                if "idms/authx" in resp.url or resp.status == 541:
                    with open("authx_response.html", "w", encoding="utf-8") as f:
                        f.write(txt)

        page.on("response", lambda r: asyncio.create_task(log_bad_response(r)))

        await page.goto("https://www.apple.com/jp/", wait_until="domcontentloaded")
        # 走到你触发订单登录的步骤（示例：点 bag -> orders）
        await page.locator("#globalnav-menubutton-link-bag").click()
        await page.wait_for_timeout(500)
        await page.get_by_role("link", name="ご注文").click()

        # 给它时间触发请求
        await page.wait_for_timeout(8000)

        await context.close()
        await browser.close()

await debug_541()

In [3]:
import os, re
from playwright.async_api import async_playwright

def safe_dir(name: str) -> str:
    return re.sub(r"[^a-zA-Z0-9._-]+", "_", name)[:60]

async def bootstrap_login(apple_id: str):
    profile_dir = os.path.join("pw_profiles", safe_dir(apple_id))
    os.makedirs(profile_dir, exist_ok=True)

    async with async_playwright() as p:
        # 用你本机安装的 Chrome，而不是 Playwright 自带 Chromium
        context = await p.chromium.launch_persistent_context(
            user_data_dir=profile_dir,
            channel="chrome",
            headless=False,
            locale="ja-JP",
            viewport={"width": 1280, "height": 840},
        )

        page = context.pages[0] if context.pages else await context.new_page()
        await page.goto("https://secure6.store.apple.com/jp/shop/order/list", wait_until="domcontentloaded")

        print("请在打开的页面手动完成登录（含2FA），确认看到订单列表后回到这里按回车继续。")
        input()

        await context.close()

# 用你的账号跑一次
await bootstrap_login("j5x648@icloud.com")


请在打开的页面手动完成登录（含2FA），确认看到订单列表后回到这里按回车继续。
